In [21]:
import sqlite3
import string
import pandas as pd

from scipy.signal import find_peaks
import re
from urllib.request import urlopen

from io import TextIOWrapper
import time
import os.path
from os import path
from queue import PriorityQueue
from itertools import combinations
from numpy.random import choice
import random
import sqlite3
import numpy as np


In [4]:

#actual Sqlite code for the database now
dbName = "interactionDB.sqlite3"
DEFAULT_PATH = os.path.join(os.path.dirname("../"), dbName)

def db_connect(db_path=DEFAULT_PATH):
    con = sqlite3.connect(db_path)
    return con


In [52]:

def db_connect(db_path=DEFAULT_PATH):
    con = sqlite3.connect(db_path)
    return con

def getSequences():
    #return the contents of the sequence table
    conn = db_connect()
    cur = conn.cursor()
    with conn:
        cur.execute("SELECT * FROM proteins")
        return cur.fetchall()

def getAllInteractions():
    # return the contents of the sequence table
    conn = db_connect()
    cur = conn.cursor()
    with conn:
        cur.execute("SELECT * FROM interaction")
        return cur.fetchall()



def getSubsetInteractions(idList):
    # return all interactions where both partners in idList
    conn = db_connect()
    cur = conn.cursor()
    cur.execute("SELECT * FROM interaction")
    ints = cur.fetchall()
    subset = []
    for i in range(0, len(ints)):
        int_potential = ints[i]
        if i%1000 == 0:
            print ("ON: ", i, " OUT OF ", len(ints))
            print (len(subset))
        #check each interaction partners if they are in the list of ids
        #print (int_potential)
        if int_potential[1] in idList and int_potential[2] in idList:
            subset.append(int_potential)
    return subset

def resetAllSequenceLens():
    #getting all sequences <= 2000 AA long
    allseqs = getSequences()
    ids = [x[0] for x in allseqs]
    actualSeqs = [x[1] for x in allseqs]
    lens = [len(x) for x in actualSeqs]
    conn = db_connect()
    cur = conn.cursor()
    #set all sequence lengths
    for i in range(0, len(ids)):
        updateQuery = "UPDATE proteins SET length=? WHERE id=?"
        cur.execute(updateQuery, (lens[i], ids[i]))
        conn.commit()

def proteinsInDBtoFastaForCDHit(lower, upper, saveName):
    #convertes the SQL table to a fasta with my_ids as the name per sequence to run 40% CD-HIT on
    #distinctProQuery = "SELECT DISTINCT sequence FROM proteins"
    conn = db_connect()
    cur = conn.cursor()
    #query = cur.execute(distinctProQuery)
    #fetched = query.fetchall()
    #confirm that total size of db is same as unique number of seqs
    allQuery = "SELECT * FROM proteins WHERE length BETWEEN " + str(lower) + " AND " + str(upper) #trRosetta trained on sequences <= upper and >= lower
    allQuery = cur.execute(allQuery)
    allFetched = allQuery.fetchall()
    #if len(fetched) != len(allFetched):
     #   print ("stopping, duplicate sequences in teh table")
    #else:
    print ("writing all sequences to a fasta")
    print (len(allFetched))
    #make into a large fasta file w/ CD-HIT
    with open(saveName, "w") as f:
        for result in allFetched:
            f.write(">" + str(result[0]) + "\n" + result[1] + "\n" )


def getClusterRepIDs(clusterFile):
    #each rep has a * by the name
    reps = []
    with open(clusterFile, "r") as f:
        lines = [x.strip() for x in f.readlines()]
        for l in lines:
            if "*" in l:
                half = l.split(">")[1].translate(str.maketrans('', '', string.punctuation))
                half = half.replace(" ","")
                reps.append(half)
    #change to ints
    reps = [int(x) for x in reps]
    return reps

#todo: extract ids of CD-HIT clusters
def getCDHIT(clusterFile, outputDir):
    if not os.path.isdir(outputDir):
        os.mkdir(outputDir)
    #each rep has a * by the name
    reps = []
    with open(clusterFile, "r") as f:
        lines = [x.strip() for x in f.readlines()]
        for l in lines:
            if "*" in l:
                half = l.split(">")[1].translate(str.maketrans('', '', string.punctuation))
                half = half.replace(" ","")
                reps.append(half)
    #get sequences from db
    conn = db_connect()
    cur = conn.cursor()
    match = 0
    notMatch = 0
    seqs = []
    print ('total len reps: ', len(reps))
    for id in reps:
        cur.execute("SELECT * FROM proteins WHERE id=?", (id,))
        fetched = cur.fetchall()
        if len(fetched) == 1:
            # add id to the my_ids table
            # print ("MATCHED")
            # id (generates automatically), database = str, database_name = str, my_id from proteins
            seqs.append(fetched[0][1])
            match += 1
            with open(outputDir + id + ".fasta", "w") as f:
                f.write(">" + str(id) + "\n" + fetched[0][1] + "\n")
        else:
            # print ("OVER MATCHED")
            notMatch += 1
    print ("found: ", match)
    print ("not found: ", notMatch)
    return seqs


#todo: extract ids of CD-HIT clusters
def getCDHITOneFasta(clusterFile, fastaName):
    #each rep has a * by the name
    reps = []
    with open(clusterFile, "r") as f:
        lines = [x.strip() for x in f.readlines()]
        for l in lines:
            if "*" in l:
                half = l.split(">")[1].translate(str.maketrans('', '', string.punctuation))
                half = half.replace(" ","")
                reps.append(half)
    #get sequences from db
    conn = db_connect()
    cur = conn.cursor()
    match = 0
    notMatch = 0
    seqs = []
    print ('total len reps: ', len(reps))
    outFasta = open(fastaName, "w")
    for id in reps:
        cur.execute("SELECT * FROM proteins WHERE id=?", (id,))
        fetched = cur.fetchall()
        if len(fetched) == 1:
            # add id to the my_ids table
            # print ("MATCHED")
            # id (generates automatically), database = str, database_name = str, my_id from proteins
            seqs.append(fetched[0][1])
            match += 1
            outFasta.write(">" + str(id) + "\n" + fetched[0][1] + "\n")
        else:
            # print ("OVER MATCHED")
            notMatch += 1
    print ("found: ", match)
    print ("not found: ", notMatch)
    outFasta.close()
    return seqs


def getTransformerMeanSequenceOutput(seq):
    #from github: pooled_output is *not* trained for the transformer
    model = ProteinBertModel.from_pretrained('bert-base')
    tokenizer = TAPETokenizer(vocab='iupac')  # iupac is the vocab for TAPE models, use unirep for the UniRep model
    token_ids = torch.tensor([tokenizer.encode(seq)])
    output = model(token_ids)
    #average embedding for the bert model, average along sequence as recommendd
    #output shape is (1,768)
    return torch.mean(output[0],1)

def getUniRepPooledSequence(sequence):
    #return
    model = UniRepModel.from_pretrained('babbler-1900')
    token_ids = torch.tensor([tokenizer.encode(sequence)])
    output = model(token_ids)
    return output[1]


def retrieveOrigNames(namesDone):
    #retrieve count of proteins in HuRI
    con = db_connect()
    cur = con.cursor()
    huRIMappedNames = []
    for i in range(0, len(namesDone)):
        name = namesDone[i]
        if i % 1000 == 0:
            print ("ON ", i , " OUT OF ", len(namesDone))
        lookupQ = cur.execute("SELECT * FROM id_map WHERE database=? AND my_id=?", ("HuRI", name))
        fetched = lookupQ.fetchall()
        if len(fetched) != 0:
            #print ("found match in HuRI!!!")
            huRIMappedNames.append(fetched[0][2])
    #print (len(huRIMappedNames))
    return huRIMappedNames

def overlapTwoLists(list1, list2):
    intersection = set(list1).intersection(set(list2))
    return list(intersection)

def lookAtPositivesAndNegativesCurrent(listIDs):
    # looking at the number of positive examples with the cluster reps
    cdHitSeqs = getClusterRepIDs("embedPPI_50_2000_70_Cutoff.clstr")  # sequence IDs in the DB
    # get all interactions
    subsetGoodFor = getSubsetInteractions(cdHitSeqs)
    positives = [x for x in subsetGoodFor if x[8] != 1]
    print(len(positives))  # 279775
    negatives = [x for x in subsetGoodFor if x[8] == 1]
    print(len(negatives))  # 918
    return positives, negatives



def lookAtIntersection(huRIMappedNames):
    #making negatives for HuRI interactions
    print ('start ret')
    in05 = open("./HuRI/HuRI_05_proteins.txt", "r")
    in05 = [x.strip() for x in in05.readlines()]
    intersect1 = overlapTwoLists(huRIMappedNames, in05)
    print(len(intersect1))
    print(len(in05))
    in14 = open("./HuRI/HuRI_14_proteins.txt", "r")
    in14 = [x.strip() for x in in14.readlines()]
    intersect2 = overlapTwoLists(huRIMappedNames, in14)
    print(len(intersect2))
    print(len(in14))
    in19 = open("./HuRI/HuRI_HuRI_proteins.txt", "r")
    in19 = [x.strip() for x in in19.readlines()]
    intersect3 = overlapTwoLists(huRIMappedNames, in19)
    print(len(intersect3))
    print(len(in19))

    #all HuRi pros
    allHuRI = list(set(in05 + in14 + in19)) #all proteins
    #print (allHuRI)
    allIntersect = list(set(intersect1 + intersect2 + intersect3))
    print (len(allIntersect))
    # retrieve HuRI associated names to construct HuRI negatives
    # print (namesDone)
    return list(set(intersect1)), list(set(intersect2)), list(set(intersect3))

def addEqualNumberNegativesToDB(cdHitSeqs, N):
    #adds N negative HuRI interactiosn to the database based on number of positive interactions in the dataset
    countAdded = 0
    conn = db_connect()
    cur = conn.cursor()
    intersect1, intersect2, intersect3 = lookAtIntersection(cdHitSeqs)
    int1_weight = len(intersect1)
    int2_weight = len(intersect2)
    int3_weight = len(intersect3)
    total_weight = int1_weight + int2_weight + int3_weight
    while countAdded < N: #keep adding until enough are reached
        #select all-by-all
        setIDs = random.choices([intersect1, intersect2, intersect3], weights = [int1_weight/total_weight, int2_weight/total_weight, int3_weight/total_weight])
        #select two ids from set
        #print (setIDs)
        #print (len(setIDs))
        #print (setIDs)
        twoIDs = np.random.choice(setIDs[0], size = 2, replace=True)
        #see if can get ids
        cur.execute("SELECT * FROM id_map WHERE database=? AND database_name=?", ("HuRI", twoIDs[0]))
        fetched_a = cur.fetchall()
        cur.execute("SELECT * FROM id_map WHERE database=? AND database_name=?", ("HuRI", twoIDs[1]))
        fetched_b = cur.fetchall()
        ids = []
        ids.append(fetched_a[0][3])
        ids.append(fetched_b[0][3])
        ids.sort()
        #print (ids)
        if len(fetched_a) == 1 and len(fetched_b) == 1:
            # print (fetched_a, fetched_b)
            # prep to add to interactions
            # try to see if it already exists
            search_interactions = "SELECT * FROM interaction WHERE protein_1=? AND protein_2=?"
            cur.execute(search_interactions, (ids[0], ids[1]))
            interaction_search = cur.fetchall()
            if len(interaction_search) == 0:
                id_map_sql = "INSERT INTO interaction (protein_1, protein_2, huri_negative) VALUES (?,?,?)"
                cur.execute(id_map_sql, (ids[0], ids[1], True))
                conn.commit()
                countAdded += 1
        if countAdded % 1000 == 0:
            print("on: ", countAdded)

In [35]:
cdHitSeqs = getClusterRepIDs("../embedPPI_50_2000_70_Cutoff.clstr") #sequence IDs in the DB


In [11]:
huRIMappedNames = retrieveOrigNames(cdHitSeqs)
print(len(huRIMappedNames))

ON  0  OUT OF  16614
ON  1000  OUT OF  16614
ON  2000  OUT OF  16614
ON  3000  OUT OF  16614
ON  4000  OUT OF  16614
ON  5000  OUT OF  16614
ON  6000  OUT OF  16614
ON  7000  OUT OF  16614
ON  8000  OUT OF  16614
ON  9000  OUT OF  16614
ON  10000  OUT OF  16614
ON  11000  OUT OF  16614
ON  12000  OUT OF  16614
ON  13000  OUT OF  16614
ON  14000  OUT OF  16614
ON  15000  OUT OF  16614
ON  16000  OUT OF  16614
6655


In [53]:
#lookAtIntersection(cdHitSeqs)
addEqualNumberNegativesToDB(huRIMappedNames, 279058) #need exactly 279775 - 100 

start ret
805
1178
2711
4078
5398
7307
5901
on:  1000
on:  2000
on:  3000
on:  4000
on:  5000
on:  6000
on:  7000
on:  8000
on:  9000
on:  10000
on:  11000
on:  12000
on:  13000
on:  14000
on:  15000
on:  16000
on:  17000
on:  18000
on:  19000
on:  20000
on:  21000
on:  22000
on:  23000
on:  24000
on:  25000
on:  26000
on:  27000
on:  28000
on:  29000
on:  30000
on:  31000
on:  32000
on:  33000
on:  34000
on:  35000
on:  36000
on:  37000
on:  38000
on:  39000
on:  40000
on:  41000
on:  42000
on:  43000
on:  44000
on:  45000
on:  46000
on:  47000
on:  48000
on:  49000
on:  50000
on:  51000
on:  52000
on:  53000
on:  54000
on:  55000
on:  56000
on:  57000
on:  58000
on:  59000
on:  60000
on:  61000
on:  62000
on:  63000
on:  64000
on:  65000
on:  66000
on:  67000
on:  68000
on:  69000
on:  70000
on:  71000
on:  72000
on:  73000
on:  74000
on:  75000
on:  76000
on:  77000
on:  78000
on:  79000
on:  80000
on:  81000
on:  82000
on:  83000
on:  84000
on:  85000
on:  86000
on:  87000
on:  880

In [44]:
conn = db_connect()
cur = conn.cursor()
del1 = "DELETE FROM interaction WHERE huri_negative=1"
cur.execute(del1)
conn.commit()